In [12]:
from dotenv import load_dotenv
import os
import pandas as pd
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI


In [9]:
df = pd.read_csv("spotify.csv", encoding='ISO-8859-1')
print(df.shape)
print(df.columns.tolist())

(953, 24)
['track_name', 'artist(s)_name', 'artist_count', 'released_year', 'released_month', 'released_day', 'in_spotify_playlists', 'in_spotify_charts', 'streams', 'in_apple_playlists', 'in_apple_charts', 'in_deezer_playlists', 'in_deezer_charts', 'in_shazam_charts', 'bpm', 'key', 'mode', 'danceability_%', 'valence_%', 'energy_%', 'acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%']


In [18]:
# engine = create_engine("sqlite:///spotify.db")
# df.to_sql("spotify", engine, index=False)
df.head()

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,144,A,Minor,65,23,80,14,63,11,6


In [11]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM spotify WHERE released_year = 2019;")

sqlite
['spotify']


'[(\'Cruel Summer\', \'Taylor Swift\', 1, 2019, 8, 23, 7858, 100, \'800840817\', 116, 207, \'125\', 12, \'548\', 170, \'A\', \'Major\', 55, 58, 72, 11, 0, 11, 15), (\'Blinding Lights\', \'The Weeknd\', 1, 2019, 11, 29, 43899, 69, \'3703895074\', 672, 199, \'3,421\', 20, None, 171, \'C#\', \'Major\', 50, 38, 80, 0, 0, 9, 7), (\'Watermelon Sugar\', \'Harry Styles\', 1, 2019, 11, 17, 21915, 34, \'2322580122\', 437, 115, \'1,212\', 12, None, 95, None, \'Major\', 55, 56, 82, 12, 0, 34, 5), (\'Those Eyes\', \'New West\', 1, 2019, 5, 10, 1507, 14, \'411747614\', 24, 71, \'44\', 1, \'195\', 120, \'E\', \'Major\', 60, 24, 35, 73, 0, 31, 3), (\'Under The Influence\', \'Chris Brown\', 1, 2019, 10, 4, 3859, 26, \'929964809\', 133, 181, \'3\', 0, None, 117, \'A\', \'Minor\', 73, 31, 69, 6, 0, 11, 4), ("Car\'s Outside", \'James Arthur\', 1, 2019, 10, 18, 794, 10, \'265882712\', 38, 25, \'61\', 0, \'263\', 150, \'A\', \'Major\', 34, 24, 56, 4, 0, 11, 3), (\'Circles\', \'Post Malone\', 1, 2019, 8, 30,

In [20]:
llm = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0)
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [23]:
agent_executor.invoke({"input": "plot a heatmap between the columns", "context": "spotify"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


spotify
Invoking: `sql_db_schema` with `{'table_names': 'spotify'}`



CREATE TABLE spotify (
	track_name TEXT, 
	"artist(s)_name" TEXT, 
	artist_count BIGINT, 
	released_year BIGINT, 
	released_month BIGINT, 
	released_day BIGINT, 
	in_spotify_playlists BIGINT, 
	in_spotify_charts BIGINT, 
	streams TEXT, 
	in_apple_playlists BIGINT, 
	in_apple_charts BIGINT, 
	in_deezer_playlists TEXT, 
	in_deezer_charts BIGINT, 
	in_shazam_charts TEXT, 
	bpm BIGINT, 
	"key" TEXT, 
	mode TEXT, 
	"danceability_%" BIGINT, 
	"valence_%" BIGINT, 
	"energy_%" BIGINT, 
	"acousticness_%" BIGINT, 
	"instrumentalness_%" BIGINT, 
	"liveness_%" BIGINT, 
	"speechiness_%" BIGINT
)

/*
3 rows from spotify table:
track_name	artist(s)_name	artist_count	released_year	released_month	released_day	in_spotify_playlists	in_spotify_charts	streams	in_apple_playlists	in_apple_charts	in_deezer_playlists	in_deezer_charts	in_

{'input': 'plot a heatmap between the columns',
 'context': 'spotify',
 'output': "To plot a heatmap between the columns, we would typically select numerical columns that represent different dimensions or features of the data. Based on the schema of the `spotify` table, we have several numerical columns that could be interesting for a heatmap, such as `released_year`, `artist_count`, `in_spotify_playlists`, `in_spotify_charts`, `in_apple_playlists`, `in_apple_charts`, `in_deezer_charts`, `bpm`, `danceability_%`, `valence_%`, `energy_%`, `acousticness_%`, `instrumentalness_%`, `liveness_%`, and `speechiness_%`.\n\nHowever, as an AI developed to interact with a SQL database, I'm unable to directly plot heatmaps or any other visualizations. To create a heatmap, you would typically use a programming language like Python with libraries such as Matplotlib and Seaborn, which are designed for data visualization.\n\nIf you're interested in obtaining specific data from the `spotify` table to cre